In [1]:
parameter = "pr"
# "tas", "pr", "huss", "rlds",  "rsds", "sfcwind"

run, load_run_number = 0, 7             #  0:"single_run"  1:"keras_tuner" 2:"load_best_model"
scenario_option = 0
label = 0
colab = 0
model_choice = "tf"
percentage = 1
epochs = 2
batch_size=32
percentage_limit=0.35
pixel_interpolation_limit=50
# desired_zero_array = []
divide_latitude_in_these_many_parts = 5  #latitude has 10 values
divide_longitude_in_these_many_parts = 6 # longitude has 14 values


if scenario_option == 0:
    scenario = "historical"
elif scenario_option ==1:
    scenario = "rcp26"
elif scenario_option ==2:
    scenario = "rcp45"
elif scenario_option ==3:
    scenario = "rcp85"


# Configurations

In [2]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import plotly.express as px
import math
import time
from sklearn.linear_model import LinearRegression
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from kerastuner.tuners import RandomSearch
import logging
import os
import sys
from contextlib import contextmanager


C:\Users\suvam\AppData\Local\Temp\ipykernel_10588\3557830353.py:15: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
# for the WSL conda environment
# 1. python kernel is named wslminiconda3 (Python 3.11.14)
# 2. the environment is named base and is in the directory /home/arhab/wslminiconda3
# 3. Do not use the python kernel named base

In [4]:
def get_next_folder_name(base_folder):
    run_number = 1
    while True:
        folder_name = os.path.join(base_folder, f"run_{run_number}")
        if not os.path.exists(folder_name):
            return folder_name
        run_number += 1

# Base folder directory
base_directory = r"D:\thesis_data\notebooks\model_runs"

# Get the next folder name
folder_name = get_next_folder_name(base_directory)

# Create the folder
os.makedirs(folder_name, exist_ok=True)
run_number = f"run_{folder_name.split('_')[3]}"
print(run_number)

run_27


In [5]:
# Set up logging and save the log file in the folder
log_file = os.path.join(folder_name, 'log.txt')
logger = logging.getLogger(folder_name)
logger.setLevel(logging.INFO)

# Create a file handler and set the formatter
file_handler = logging.FileHandler(log_file)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

# Add the file handler to the logger
logger.addHandler(file_handler)

# Create a stream handler to display log messages in Jupyter Notebook console
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

# Log information
logger.info("Logging information check")
logger.info(run_number)

2024-01-18 13:00:25,255 - INFO - Logging information check
2024-01-18 13:00:25,257 - INFO - run_27


In [6]:
logger.info(f"run = {run}   =>  0:single_run  1:keras_tuner 2:load_best_model")
logger.info(f"model_choice ={model_choice}")
logger.info(f"percentage = {percentage}")
logger.info(f"epochs = {epochs}")
logger.info(f"batch_size = {batch_size}")
logger.info(f"percentage_limit = {percentage_limit}")
logger.info(f"pixel_interpolation_limit = {pixel_interpolation_limit}")

2024-01-18 13:00:25,270 - INFO - run = 0   =>  0:single_run  1:keras_tuner 2:load_best_model
2024-01-18 13:00:25,271 - INFO - model_choice =tf
2024-01-18 13:00:25,273 - INFO - percentage = 1
2024-01-18 13:00:25,275 - INFO - epochs = 2
2024-01-18 13:00:25,277 - INFO - batch_size = 32
2024-01-18 13:00:25,278 - INFO - percentage_limit = 0.35
2024-01-18 13:00:25,280 - INFO - pixel_interpolation_limit = 50


In [7]:
import contextlib
import sys

@contextlib.contextmanager
def stdout_redirected(new_stdout):
    save_stdout = sys.stdout
    sys.stdout = new_stdout
    try:
        yield None
    finally:
        sys.stdout = save_stdout


In [8]:
if colab == 1:
    # !pip install cftime
    from google.colab import drive
    drive.mount('/content/drive')
    import cftime
    %run "/content/drive/My Drive/Colab Notebooks/main/functions.ipynb"
else: 
    %run "functions.ipynb"

# Data Loading

In [9]:
# Step 1: Load NDSI Labels
if colab !=1:
  ndsi_ds = xr.open_mfdataset(r"D:\thesis_data\new_cordex_data\cropped_data\label.nc")

if colab == 1:
  ndsi_ds = xr.open_mfdataset(f'/content/drive/My Drive/Colab Notebooks/cropped_data/label.nc')

# ndsi_ds = ndsi_ds.sel(time=slice("2001-01-01", "2001-03-02"))
filtered_dates, nan_interpolation_df = get_filtered_dates_for_ndsi(percentage_limit=percentage_limit, pixel_interpolation_limit=pixel_interpolation_limit)
ndsi_ds = ndsi_ds.sel(time=filtered_dates)
selected_dates = get_dates(ndsi_ds)
ndsi_labels = ndsi_ds['NDSI_Snow_Cover'].values
ndsi_ds = ndsi_ds.interpolate_na(dim='lon', method='linear',  max_gap=8, use_coordinate=False)
ndsi_ds = ndsi_ds.interpolate_na(dim='lat', method='linear', max_gap=5, use_coordinate=False)

ndsi_ds['time'] = xr.DataArray(ndsi_ds['time'].values.astype('datetime64[ns]'), dims='time', attrs=ndsi_ds['time'].attrs)
ndsi_ds.close()

In [10]:
# Step 2: Load Data
# Load climate variables
climate_vars = ["tas", "pr", "huss", "rlds",  "rsds", "sfcWind"]
data = []

for parameter in climate_vars:
    scenario = "historical"

    if colab == 0:
      ds = xr.open_mfdataset(rf'D:\thesis_data\new_cordex_data\cropped_data\historical\{parameter}_historical.nc') #fixxx
    if colab == 1:
      ds = xr.open_mfdataset(f'/content/drive/My Drive/Colab Notebooks/cropped_data/historical/{parameter}_{scenario}.nc')


    # ds = ds.sel(time=slice("2001-01-01", "2018-31-02"))
    ds = ds.sel(time=selected_dates)
    data.append(ds[parameter].values)
    ds.close()

data = np.array(data)

# Conversion to pandas from xarray

In [11]:
# Step 5: Flatten Data
n_time_steps = data[0].shape[0]
n_lat, n_lon = data[0].shape[1], data[0].shape[2]
logger.info("for feature")
logger.info(f"{n_time_steps} , {n_lat}, {n_lon}")

n_time_steps = ndsi_labels.shape[0]
n_lat, n_lon = ndsi_labels.shape[1], ndsi_labels.shape[2]
logger.info("for label")
logger.info(f"{n_time_steps} , {n_lat}, {n_lon}")


2024-01-18 13:00:28,165 - INFO - for feature
2024-01-18 13:00:28,166 - INFO - 3474 , 10, 14
2024-01-18 13:00:28,167 - INFO - for label
2024-01-18 13:00:28,169 - INFO - 3474 , 10, 14


In [12]:
temp0 = to_array(data, 0)
temp1 = to_array(data, 1)
temp2 = to_array(data, 2)
temp3 = to_array(data, 3)
temp4 = to_array(data, 4)
temp5 = to_array(data, 5)

ndsi_array = to_array_ndsi(ndsi_labels)

In [13]:
dates = ds["time"].values
lats = ndsi_ds["lat"].values
lons = ndsi_ds["lon"].values

#********** Dates *************
dates_array_to_append = []
for a in range(len(dates)):
    x = dates[a]
    for b in range(140):
        dates_array_to_append.append(x)
dates_array_to_append = np.array(dates_array_to_append)

#********** Latitude *************
lats_array_to_append = []
for c in range(len(dates)):
    for a in range(len(lats)):
        x = lats[a]
        for b in range(len(lons)):
            lats_array_to_append.append(x)
lats_array_to_append = np.array(lats_array_to_append)

#********** Longitude *************
lons_array_to_append = []
for b in range(len(dates)*len(lats)):
    for a in range(len(lons)):
        x = lons[a]
        lons_array_to_append.append(x)

lons_array_to_append = np.array(lons_array_to_append)


logger.info(f"Length of Dates: {len(dates_array_to_append)}")
logger.info(f"Length of Latitude: {len(lats_array_to_append)}")
logger.info(f"Length of Longitude: {len(lons_array_to_append)}")

2024-01-18 13:00:30,640 - INFO - Length of Dates: 486360
2024-01-18 13:00:30,652 - INFO - Length of Latitude: 486360
2024-01-18 13:00:30,653 - INFO - Length of Longitude: 486360


# Data cleaning, feature engineering with pandas

In [14]:
dict_temp = {
    "Date": dates_array_to_append,
    "Latitude": lats_array_to_append,
    "Longitude": lons_array_to_append,
    climate_vars[0]: temp0,
    climate_vars[1]: temp1,
    climate_vars[2]: temp2,
    climate_vars[3]: temp3,
    climate_vars[4]: temp4,
    climate_vars[5]: temp5,
    "ndsi1": ndsi_array
}


# Create the DataFrame
df = pd.DataFrame(dict_temp)
df['Latitude'] = df['Latitude'].round(2)
df['Longitude'] = df['Longitude'].round(2)
df, drop_nan = df.dropna(ignore_index=False), "yes"


In [15]:
# df = df.reset_index()
df["Month"] = df['Date'].dt.month
df['month_sin'] = np.sin(2 * np.pi * df['Month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['Month']/12)

df['week_number'] = df["Date"].dt.isocalendar().week

In [16]:
# Original one-hot encoding
# lat_one_hot_encoded = pd.get_dummies(df['Latitude'], prefix='Latitude', dtype=int)
# lon_one_hot_encoded = pd.get_dummies(df['Longitude'], prefix='Longitude', dtype=int)

# df = pd.concat([df, lat_one_hot_encoded, lon_one_hot_encoded], axis=1)


In [17]:
# # One-hot encoding
# #Latitude
# df['Lat_46_to_46d36'] = (df['Latitude'].between(46, 46.36)).astype(int)
# df['Lat_46d37_to_46d69'] = (df['Latitude'].between(46.36, 46.70)).astype(int)
# df['Lat_46d8_to_47d14'] = (df['Latitude'].between(46.70, 47.15)).astype(int)

# #Longitude
# df['Lon_10_to_10d67'] = (df['Longitude'].between(10, 10.67)).astype(int)
# df['Lon_10d67_to_11d32'] = (df['Longitude'].between(10.67, 11.32)).astype(int)
# df['Lon_11d32_to_11d81'] = (df['Longitude'].between(11.33, 11.82)).astype(int)
# df['Lon_11d81_to_12d29'] = (df['Longitude'].between(11.83, 12.29)).astype(int)


In [18]:
lats = np.unique(df.Latitude.values)
lons = np.unique(df.Longitude.values)

latitude_bins = divide_range(lats[0], lats[-1], divide_latitude_in_these_many_parts)   #10 values 
longitude_bins = divide_range(lons[0], lons[-1], divide_longitude_in_these_many_parts) #14 values

df['Latitude_Group'] = pd.cut(df['Latitude'], bins=latitude_bins, labels=False)
df['Longitude_Group'] = pd.cut(df['Longitude'], bins=longitude_bins, labels=False)

# Apply one-hot encoding
latitude_dummies = pd.get_dummies(df['Latitude_Group'], prefix='Latitude', dtype=int)
longitude_dummies = pd.get_dummies(df['Longitude_Group'], prefix='Longitude', dtype=int)

# Concatenate one-hot encoded columns with the original DataFrame
df = pd.concat([df, latitude_dummies, longitude_dummies], axis=1)
temp = df
df = df.drop(["Latitude_Group",	"Longitude_Group"], axis =1)

temp = temp[temp["Date"] == temp.Date[1]]
grouped_latitudes = temp.groupby('Latitude_Group')['Latitude'].unique().to_dict()
logger.info("Latitude Group")
logger.info("{\n" + ",\n".join(f" {key}: {value}" for key, value in grouped_latitudes.items()) + "\n}")

grouped_longitudes = temp.groupby('Longitude_Group')['Longitude'].unique().to_dict()
logger.info("Longitude Group")
logger.info("{\n" + ",\n".join(f" {key}: {value}" for key, value in grouped_longitudes.items()) + "\n}")


2024-01-18 13:00:31,149 - INFO - Latitude Group
2024-01-18 13:00:31,151 - INFO - {
 0: [46.13 46.24],
 1: [46.35 46.47],
 2: [46.58 46.69],
 3: [46.8  46.92],
 4: [47.03 47.14]
}
2024-01-18 13:00:31,155 - INFO - Longitude Group
2024-01-18 13:00:31,158 - INFO - {
 0: [10.18 10.34 10.5 ],
 1: [10.66 10.82],
 2: [10.99 11.15],
 3: [11.31 11.47],
 4: [11.64 11.8 ],
 5: [11.96 12.12 12.28]
}


In [19]:
#bookmark
print(df.columns)
df

Index(['Date', 'Latitude', 'Longitude', 'tas', 'pr', 'huss', 'rlds', 'rsds',
       'sfcWind', 'ndsi1', 'Month', 'month_sin', 'month_cos', 'week_number',
       'Latitude_0', 'Latitude_1', 'Latitude_2', 'Latitude_3', 'Latitude_4',
       'Longitude_0', 'Longitude_1', 'Longitude_2', 'Longitude_3',
       'Longitude_4', 'Longitude_5'],
      dtype='object')


,Date,Latitude,Longitude,tas,pr,huss,rlds,rsds,sfcWind,ndsi1,...,Latitude_1,Latitude_2,Latitude_3,Latitude_4,Longitude_0,Longitude_1,Longitude_2,Longitude_3,Longitude_4,Longitude_5
0,2001-01-01 12:00:00,46.13,10.18,263.203278,1.564806e-09,0.001066,213.009583,83.646156,2.344069,62.501556,...,0,0,0,0,1,0,0,0,0,0
1,2001-01-01 12:00:00,46.13,10.34,269.678467,2.030947e-08,0.001068,211.131866,80.710426,1.355827,61.311447,...,0,0,0,0,1,0,0,0,0,0
2,2001-01-01 12:00:00,46.13,10.50,256.047241,1.650252e-08,0.000851,199.728699,79.973618,1.899092,65.980927,...,0,0,0,0,1,0,0,0,0,0
3,2001-01-01 12:00:00,46.13,10.66,260.295776,7.775169e-11,0.001006,202.456894,81.029640,3.666345,68.090210,...,0,0,0,0,0,1,0,0,0,0
4,2001-01-01 12:00:00,46.13,10.82,266.777985,2.028565e-07,0.001364,213.373550,77.965660,2.358261,53.926231,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486291,2018-12-31 12:00:00,46.69,10.34,263.931519,2.894347e-06,0.002223,222.105026,82.042076,2.121476,50.924480,...,0,1,0,0,1,0,0,0,0,0
486294,2018-12-31 12:00:00,46.69,10.82,267.040100,2.160477e-05,0.002353,205.163071,81.809784,3.399118,52.055504,...,0,1,0,0,0,1,0,0,0,0
486297,2018-12-31 12:00:00,46.69,11.31,270.152191,7.662312e-06,0.003096,218.106552,79.163933,4.733071,43.464687,...,0,1,0,0,0,0,0,1,0,0
486298,2018-12-31 12:00:00,46.69,11.47,267.166138,1.842953e-05,0.002798,232.383804,71.881706,2.661735,37.354782,...,0,1,0,0,0,0,0,1,0,0


In [20]:
df["ndsi"] = df["ndsi1"]
df= df.drop(["Month", "week_number", "ndsi1"],axis =1)

In [21]:
parameter_array = ["tas", "pr", "huss", "rlds",  "rsds", "sfcWind"]
min_max_df = pd.read_csv("./min_max_of_all_parameters.csv")
for parameter in parameter_array:
    min_val, max_val =  get_min_max(min_max_df, parameter)
    df[parameter] = (df[parameter] - min_val) / (max_val - min_val)

df['ndsi'] = df['ndsi'].apply(lambda x: x / 100 if not pd.isnull(x) else x)

In [22]:
# percentage_array= []
# actual_total_zeroes_array = []
# desired_zero_array = [25000]  # Desired number of zero values
# b = 0.01  # Initial percentage to sample

# for a in desired_zero_array:
#     a = a - 2500 
#     tolerance = 0.15 * a 

#     df["Month"] = df['Date'].dt.month
#     df_to_delete = df
#     df = df[df['Month'].isin([1, 2, 3, 12])]

#     logger.info(f"Tolerance:  {tolerance}")
#     logger.info(f"Final range of near zeores: {a +2500 - tolerance} - {a + 2500 + tolerance}")

#     while True:
#         zero_values_array = []

#         # Step 2: Iterate through each month
#         for month in range(4, 12):  # Months from April to November
#             # Step 3: Sample rows and calculate the number of near zero values
#             sampled_data = df_to_delete[(df_to_delete['Date'].dt.month == month)].sample(frac=b)
#             zero_values_array.append(np.sum((sampled_data['ndsi'] >= -0.005) & (sampled_data['ndsi'] <= 0.005)))

#         # Step 4: Check the sum of zero values
#         total_zero_values = sum(zero_values_array)

#         # Step 5: Adjust the percentage (b)
#         if total_zero_values > a + tolerance:
#             b -= 0.01
#         elif total_zero_values < a - tolerance:
#             b += 0.01
#         else:
#             break  # Convergence reached

#     # Step 6: Display the final value of b
#     logger.info(f"Final value of b: {b}")
#     logger.info(f"Total zero values: {total_zero_values+2500}")
#     percentage_array.append(b)
#     actual_total_zeroes_array.append(total_zero_values+2500)


#     # Now you can use this final value of b to sample your DataFrame
#     final_sampled_data = pd.concat([df_to_delete[(df_to_delete['Date'].dt.month == month)].sample(frac=b) for month in range(4, 12)])
#     df = pd.concat([final_sampled_data, df], ignore_index=False)
#     df = df.sort_index()

#     # the old code for the sampling has been moved to less_imp/rough

### Plotting of per month data (should be kept folded to keep everything compact)

In [24]:
# import plotly.express as px
# from plotly.subplots import make_subplots
# import plotly.graph_objs as go

# # Plot histogram for df
# fig1 = px.histogram(df, x='ndsi', nbins=100)
# fig1.update_layout(
#     xaxis_title='Value',
#     yaxis_title='Frequency',
# )

# # Plot histogram for df_to_delete
# fig2 = px.histogram(df_to_delete, x='ndsi', nbins=100)
# fig2.update_layout(
#     xaxis_title='Value',
#     yaxis_title='Frequency',
# )

# # Create subplots with two columns
# fig = make_subplots(rows=1, cols=2, subplot_titles=('Before', 'After'))
# fig.add_trace(fig1.data[0], row=1, col=2)
# fig.add_trace(fig2.data[0], row=1, col=1)

# fig.update_layout(
#     title='Before and after zero sampling in mid-year months',
#     width=1000,  # Total width of the combined plots
#     height=400,  # Height of the combined plots
# )

# plotly_image_path = os.path.join(folder_name, 'before_after_zero_sampling.png')
# fig.write_image(plotly_image_path)


# fig.show()


In [25]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'df_to_delete' is your DataFrame
# Convert 'Date' column to datetime
df_to_delete['Date'] = pd.to_datetime(df_to_delete['Date'])

# Extract month from the 'Date' column
df_to_delete['Month'] = df_to_delete['Date'].dt.month
df['Month'] = df['Date'].dt.month

# Create a new figure and subplots
fig, axs = plt.subplots(4, 6, figsize=(20, 10))  # 4 rows, 8 columns for 12 months for both DataFrames


# row = (month - 1) // 3  # Calculate the row index for the subplot
#     col = (month - 1) % 3

# Plot histograms for each month in df
for month in range(1, 13):
    row = (month - 1) // 3  # Calculate the row index for the subplot
    col = (month - 1) % 3   # Calculate the column index for the subplot
    
    # Filter data for the current month in df_to_delete
    month_data_df_to_delete = df_to_delete[df_to_delete['Month'] == month]['ndsi']
    
    # Plot histogram in the corresponding subplot for df_to_delete
    axs[row, col].hist(month_data_df_to_delete, bins=20, color='salmon', alpha=0.7)
    axs[row, col].set_title(f'Month {month} (before)')
    axs[row, col].set_xlabel('NDSI')
    axs[row, col].set_ylabel('Frequency')


# Plot histograms for each month in df_to_delete on the right
for month in range(1, 13):
    row = (month - 1) // 3  # Calculate the row index for the subplot
    col = (month - 1) % 3 + 3  # Shift to the right by 4 columns
    
    # Filter data for the current month in df
    month_data_df = df[df['Month'] == month]['ndsi']
    
    # Plot histogram in the corresponding subplot for df
    axs[row, col].hist(month_data_df, bins=20, color='skyblue', alpha=0.7)
    axs[row, col].set_title(f'Month {month} (after) | Fraction = {month_fraction_array_padded[month-1]}')
    axs[row, col].set_xlabel('NDSI')
    axs[row, col].set_ylabel('Frequency')
    
    
# # Hide empty subplots
# for i in range(12, 16):
#     fig.delaxes(axs[3, i])

df = df.drop(["Month"], axis=1)
# Adjust layout and display the subplots
plt.tight_layout()

matplotlib_image_path = os.path.join(folder_name, 'each_month_zero_cleaning.png')
plt.savefig(matplotlib_image_path)


plt.show()


NameError: name 'df_to_delete' is not defined

# Data splitting

In [26]:
# percentage = 100
number_of_rows = int(percentage/100*len(df))
logger.info(f"Number of rows: {number_of_rows}/{len(df)}")
number_of_rows

2024-01-18 13:01:04,729 - INFO - Number of rows: 3907/390708


3907

In [27]:
df_reduced = df.sample(frac=percentage/100)
df_reduced = df_reduced.sort_index()

In [28]:
ultimate_test= df_reduced[df_reduced['Date'].dt.year > 2017]
df_reduced = df_reduced.drop(ultimate_test.index)
df_reduced = df_reduced.sort_index()

In [29]:
test = df_reduced.sample(frac=0.2)
test = test.sort_index()

train = df_reduced.drop(test.index)
train = train.sort_index()

In [30]:
# X = df.iloc[:number_of_rows, 1:-1].values
# y = df.iloc[:number_of_rows, -1].values

# Step 7: Split Data
X_train, X_test = train.drop(["Date", "ndsi", "Latitude", "Longitude"], axis=1).values, test.drop(["Date", "ndsi", "Latitude", "Longitude"], axis=1).values
y_train, y_test = train["ndsi"].values, test["ndsi"].values

In [31]:
X_ultimate_test, y_ultimate_test = ultimate_test.drop(["Date", "ndsi"], axis=1).values, ultimate_test["ndsi"].values

In [32]:
temp_shape = X_train.shape[1]
# Extracting features and labels
X_train = X_train.reshape(-1, 1, temp_shape)  # Reshaping to (146246, 1, 32)
y_train = y_train.reshape(-1, 1)  # Assuming the label is in column 35

X_test = X_test.reshape(-1, 1, temp_shape)  # Reshaping to (146246, 1, 32)
y_test = y_test.reshape(-1, 1)  # Assuming the label is in column 35


# logger.infoing shapes
logger.info(f"y_train shape is: {y_train.shape}")
logger.info(f"X_train shape is: {X_train.shape}")

logger.info(f"y_test shape is: {y_test.shape}")
logger.info(f"X_test shape is: {X_test.shape}")


2024-01-18 13:01:07,952 - INFO - y_train shape is: (2946, 1)
2024-01-18 13:01:07,953 - INFO - X_train shape is: (2946, 1, 19)
2024-01-18 13:01:07,955 - INFO - y_test shape is: (737, 1)
2024-01-18 13:01:07,956 - INFO - X_test shape is: (737, 1, 19)


In [33]:
# %run "conversion_to_LSTM_format.ipynb"

# Model Architecture

## Machine learning stuff, mainly folded

In [34]:
if model_choice=="ml":
    # Step 8: Train Random Forest Model
    rf_model = RandomForestRegressor(n_estimators=100)

    start_time = time.time()
    rf_model.fit(X_train, y_train)
    end_time = time.time()
    execution_time_fitting = end_time - start_time
    logger.info(f"Time taken to fit {execution_time_fitting/60} min")
    logger.info("Fitting done")

    # Step 9: Model historical
    start_time = time.time()
    y_pred = rf_model.predict(X_test)
    end_time = time.time()
    y_ultimate_pred = rf_model.predict(X_ultimate_test)
    execution_time_prediciting = end_time - start_time

    logger.info(f"Time taken to predict {execution_time_prediciting}")


In [35]:
# # Define the model
#     model = Sequential()
#     # model.add(LSTM(128, input_shape=(1,32), activation='relu'))
#     model.add(LSTM(units=64, input_shape=(1,7), return_sequences=False, activation='relu'))
#     # model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))  # Increased neurons
#     # model.add(Dropout(0.5))
#     # # model.add(Dense(128, activation='relu'))
#     # # model.add(Dropout(0.5))
#     # model.add(Dense(64, activation='relu'))
#     # model.add(Dropout(0.5))
#     # model.add(Dense(32, activation='relu'))
#     # model.add(Dropout(0.5))
#     model.add(Dense(1, activation='linear'))  # Assuming NDSI is a continuous variable

## Deep learning stuff, the main thing 

In [36]:
if model_choice=="tf":
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, LSTM, SimpleRNN
    from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
    from tensorflow.keras import optimizers

    # Check for GPU availability
    if tf.config.list_physical_devices('GPU'):
        logger.info('GPU found. Running on GPU.')
    else:
        logger.info('No GPU found. Running on CPU.')

    # Specify GPU device if available
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)

    # Convert data to TensorFlow tensors
    X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
    y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
    X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
    y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)
    


2024-01-18 13:01:14,209 - INFO - GPU found. Running on GPU.


In [37]:
if model_choice=="tf" and run ==0:
    # Define the model
    model = Sequential()
    model.add(LSTM(units=180, input_shape=(1,temp_shape), return_sequences=True, activation='relu'))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(10, return_sequences=True))
    model.add(LSTM(5, return_sequences=False))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='linear'))  # Assuming NDSI is a continuous variable

    # Compile the model
    model.compile(optimizer="adam", loss='mean_squared_error', metrics=['mae'])

    # filepath = 'models/{epoch:02d}-{loss:.4f}-{val_loss:.4f}-{mae:.4f}-{val_mae:.4f}.hdf5'
    callbacks = [EarlyStopping(monitor='val_loss', patience=20)]
    # ModelCheckpoint(filepath, monitor='loss', save_best_only=True, mode='min')

    with open(log_file, 'a') as f:
        with stdout_redirected(f):
            model.summary()
            
            # Train the model on GPU
            start_time = time.time()
            history = model.fit(X_train_tensor, y_train_tensor, epochs=epochs, batch_size=batch_size, callbacks=callbacks, validation_split=0.2)
            end_time = time.time()
            execution_time_fitting = end_time - start_time

            # Evaluate the model on GPU
            loss, mae = model.evaluate(X_test_tensor, y_test_tensor)
            logger.info(f'Mean absolute Error on Test Set: {mae}')

    # Make predictions on GPU
    y_pred = model.predict(X_test_tensor)

    remove_eta_lines(run_number)
    model_save_path = os.path.join(folder_name, 'model.h5')
    model.save(model_save_path)

2024-01-18 13:01:29,676 - INFO - Mean absolute Error on Test Set: 0.12213493883609772


24/24 [==============================] - 2s 7ms/step


In [38]:
if model_choice=="tf" and run ==1:
    # Define a function to build the model with hyperparameters
    def build_model(hp):
        model = Sequential()
        model.add(LSTM(units=hp.Int('units_1', min_value=50, max_value=200, step=10), input_shape=(1, temp_shape), return_sequences=True, activation='relu'))
        model.add(LSTM(units=hp.Int('units_2', min_value=20, max_value=100, step=10), return_sequences=True))
        model.add(LSTM(units=hp.Int('units_3', min_value=10, max_value=50, step=10), return_sequences=True))
        model.add(LSTM(units=hp.Int('units_4', min_value=5, max_value=30, step=5), return_sequences=False))
        model.add(Dense(units=hp.Int('units_5', min_value=10, max_value=100, step=10), activation='relu'))
        model.add(Dense(1, activation='linear'))

        optimizer_choice = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd'])  # Define optimizers for regression
        if optimizer_choice == 'adam':
            optimizer = optimizers.Adam(learning_rate=hp.Choice('learning_rate_adam', values=[1e-2, 1e-3, 1e-4]))
        elif optimizer_choice == 'rmsprop':
            optimizer = optimizers.RMSprop(learning_rate=hp.Choice('learning_rate_rmsprop', values=[1e-2, 1e-3, 1e-4]))
        else:
            optimizer = optimizers.SGD(learning_rate=hp.Choice('learning_rate_sgd', values=[1e-2, 1e-3, 1e-4]))

        model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
        return model

    # Instantiate the tuner for hyperparameter search
    tuner = RandomSearch(
        build_model,
        objective='val_loss',
        max_trials=3,  # Number of hyperparameter combinations to try
        executions_per_trial=1,
        directory=folder_name,  # Directory to store the results
        project_name='regression_optimizers_tuner_2'
    )

    with open(log_file, 'a') as f:
        with stdout_redirected(f):
            # Search for the best hyperparameter configuration
            tuner.search(X_train_tensor, y_train_tensor, epochs=50, validation_split=0.2)



In [ ]:
#bookmark
if model_choice=="tf" and run ==1:
    # Get the best hyperparameters and build the final model
    best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hp)

    # Train the best model
    history = best_model.fit(X_train_tensor, y_train_tensor, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=[EarlyStopping(patience=20)])

    # Evaluate the best model
    loss, mae = best_model.evaluate(X_test_tensor, y_test_tensor)
    logger.info(f'Mean absolute Error on Test Set: {mae}')

    # Make predictions using the best model
    y_pred = best_model.predict(X_test_tensor)


In [ ]:
from keras.models import load_model
if model_choice=="tf" and run ==2:
    # Provide the directory path where checkpoint files or model information is stored

    # Load the model
    loaded_model = load_model(rf'D:\thesis_data\notebooks\model_runs\run_{load_run_number}\model.h5')

    # Train the best model (assuming X_train and y_train are defined)
    # history = loaded_model.fit(X_train_tensor, y_train_tensor, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=[EarlyStopping(patience=20)])

    # Evaluate the best model
    loss, mae = loaded_model.evaluate(X_test_tensor, y_test_tensor)
    logger.info(f'Mean absolute Error on Test Set: {mae}')

    # Make predictions using the best model
    y_pred = loaded_model.predict(X_test_tensor)


In [ ]:
if model_choice=="tf":
    # Plot the training and validation loss
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Mean Squared Error')
    plt.legend()
    
    matplotlib_image_path = os.path.join(folder_name, 'loss_vs_epoch_graph.png')
    plt.savefig(matplotlib_image_path)
    
    plt.show()

In [ ]:
import math
from sklearn.metrics import  mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

if model_choice == "tf":
    y_pred = y_pred.flatten()
    y_test = y_test.flatten()


mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# nse_value = nse(y_pred, y_test)

mse_value = mean_squared_error(y_test,y_pred)
rmse_value = math.sqrt(mse_value)
n_rmse_value = calculate_n_rmse(y_test, y_pred)

pbias_value = pbias(y_test, y_pred)

temp_percentage = round(number_of_rows*100/len(df),2)

In [ ]:
logger.info(f"{number_of_rows}/{len(df)} = {temp_percentage} % \n")
# logger.info(f"Time taken to fit {execution_time_fitting/60} min")
# logger.info(f"NSE value: {nse_value}")
logger.info(f'R2 Score: {r2}')
# logger.info(f'Self defined R2 Score: {self_defined_r2} \n')
logger.info(f"P-bias value: {pbias_value} \n")

# logger.info(f"MSE value: {mse_value}")
logger.info(f"RMSE value: {rmse_value}")
logger.info(f"N-RMSE value: {n_rmse_value}")

# Plotting

In [ ]:
# y_pred = y_pred.flatten()
comparison_df = pd.DataFrame({
    "Date" : test["Date"].values,
    "Latitude" : test["Latitude"].values,
    "Longitude" : test["Longitude"].values,
    "y_test": y_test,
    "y_pred": y_pred.flatten()
    })
comparison_df['Date'] = pd.to_datetime(comparison_df['Date'])

df_image_path = os.path.join(folder_name, 'test_vs_predicted.csv')
comparison_df.to_csv(df_image_path)

In [ ]:
comparison_df, scale = comparison_df.sort_values(by='Date'), "Original without mean"
fig = px.scatter(comparison_df, x="Date" , y=["y_pred","y_test"])
fig.update_layout(title=f'Predicted vs Observed | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, 'Original_without_mean.png')
fig.write_image(plotly_image_path)

fig

In [ ]:
comparison_df_1 = comparison_df[(comparison_df['Latitude'] ==  46.58) & (comparison_df['Longitude'] == 11.15)]
comparison_df_1, scale = comparison_df_1.sort_values(by='Date'), "Centre without mean"
fig = px.scatter(comparison_df, x="Date" , y=["y_pred","y_test"])
fig.update_layout(title=f'Predicted vs Observed | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, 'Centre_without_mean.png')
fig.write_image(plotly_image_path)

fig

In [ ]:
comparison_df['Date'], scale = comparison_df['Date'].dt.strftime('%d-%m-%Y'), "Average Daily"
comparison_df= comparison_df.groupby('Date')[["y_test", "y_pred"]].mean().reset_index()
comparison_df['Date'] = pd.to_datetime(comparison_df['Date'], format='%d-%m-%Y')
comparison_df = comparison_df.sort_values(by='Date')
# comparison_df

In [ ]:
fig = px.scatter(comparison_df, x="Date" , y=["y_pred","y_test"])
fig.update_layout(title=f'Predicted vs Observed | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, 'Average daily.png')
fig.write_image(plotly_image_path)
fig

In [ ]:
fig = px.line(comparison_df, x="Date" , y=["y_pred","y_test"])
fig.update_layout(title=f'Predicted vs Observed | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, 'Average daily (line).png')
fig.write_image(plotly_image_path)
fig

In [ ]:
comparison_df['Date'], scale = comparison_df['Date'].dt.strftime('%m-%Y'), "Average monthly"
comparison_df= comparison_df.groupby('Date')[["y_test", "y_pred"]].mean().reset_index()
comparison_df['Date'] = pd.to_datetime(comparison_df['Date'])
comparison_df = comparison_df.sort_values(by='Date')

In [ ]:
fig = px.line(comparison_df, x="Date" , y=["y_pred","y_test"])
fig.update_layout(title=f'Predicted vs Observed | {scale}', width = 1800, height = 400)


plotly_image_path = os.path.join(folder_name, 'Average monthly.png')
fig.write_image(plotly_image_path)

fig

In [ ]:
# # for ultimate testing

# y_ultimate_pred = y_ultimate_pred.flatten()
# comparison_df = pd.DataFrame({
#     "Date" : ultimate_test["Date"].values,
#     "y_ultimate_test": y_ultimate_test,
#     "y_ultimate_pred": y_ultimate_pred
#     })
# # comparison_df['Date'] = pd.to_datetime(comparison_df['Date'])
# # comparison_df['Date'] = comparison_df['Date'].dt.strftime('%m-%Y')


# comparison_df= comparison_df.groupby('Date')[["y_ultimate_test", "y_ultimate_pred"]].mean().reset_index()
# comparison_df['Date'] = pd.to_datetime(comparison_df['Date'])
# comparison_df = comparison_df.sort_values(by='Date')
# comparison_df


# fig = px.line(comparison_df, x="Date" , y=["y_ultimate_pred","y_ultimate_test"])
# fig.update_layout(title=f'Predicted vs Observed')